In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from morphomics.io.io import save_obj, load_obj
import torch as th

In [48]:
path_to_pi = "results/vectorization/Morphomics.PID_v1_l.pi_100_filtered"
mf = load_obj(path_to_pi)
pis = np.stack(mf['pi'])

In [49]:
mf_kxa = mf[mf['Model'].isin(['1xSaline_4h', '1xKXA _4h'])]
pis_kxa = np.stack(mf_kxa['pi'])

In [50]:
path_to_model_params = '/home/thomas/Desktop/morphomics_exp_v1/results/model/vae'
vae_params = load_obj(path_to_model_params)

/home/thomas/miniconda3/envs/morpho_exp/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/thomas/miniconda3/envs/morpho_exp/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [56]:
from morphomics.persistent_homology.pi_transformations import filter_pi_list
pis_filtered, tokeep = filter_pi_list(pis_kxa, tokeep = None, std_threshold = 0.1e-4)


3649 pixels to keep over 10000


In [ ]:
pis = np.vstack(mf['pi'])
print(mf.keys())
# Filtering
threshold = 1e-5

X_filter, tokeep = local_utils.filter_image(pis, std_threshold=threshold)

In [12]:
threshold = vae_params['threshold']
standardize = vae_params['standardizer']
fit_pca = vae_params['pca']
trained_model = vae_params['vae']

In [15]:
fit_pca.components_.shape

(50, 3681)

In [ ]:
# STD
X_filter_st = standardize.transform(X_filter)
# PCA
X_pca = fit_pca.transform(X = X_filter_st)
X_pca = th.tensor(X_pca, dtype=th.float32)


In [ ]:
pred, z_mean, z_log_var, mse = train_test.vae_test(data = X_pca,
                                                model = trained_model, 
                                                sample_size = 3,
                                               )

In [ ]:
import plotly.express as px
import numpy as np

# Create a DataFrame for easier plotting (optional, but recommended)
import pandas as pd
conditions = mf[['Model', 'Sex']].apply(lambda x: '-'.join(x), axis=1)
mf['conditions'] = conditions
labels = sorted(conditions.unique())

colors = {}
rgb_list = [
    'rgb(255, 50, 255)',   # Magenta shade
    'rgb(50, 255, 255)',   # Cyan shade
    'rgb(130, 130, 130)',  # Light grey shade
    'rgb(20, 20, 20)'      # Dark grey shade
]
for label, rgb in zip(labels, rgb_list):
    colors[label] = rgb

df = pd.DataFrame(z_mean[:,[0,1]], columns=['zx', 'zy'])
df['Label'] = mf['conditions']

# Plot using plotly.express
fig = px.scatter(df, x='zx', y='zy', color='Label', 
                 title="2D Scatter Plot with Labels",
                 labels={'Label': 'Condition'},
                 color_discrete_map=colors  # Apply custom color map
)
average_vectors = df.groupby('Label')[['zx', 'zy']].median().reset_index()

import plotly.graph_objects as go

fig.add_trace(
    go.Scattergl(
        x=average_vectors['zx'], 
        y=average_vectors['zy'], 
        mode='markers+text',  # Include text labels
        marker=dict(
            size=15,  # Larger marker size
            color=average_vectors['Label'].map(colors),  # Map color based on condition
            line=dict(width=2, color='black')  # Optional: black outline for clarity
        ),
        text=None,  # Set the text to the extracted labels
        textposition='top center',  # Position of the labels
        name='Median',
    )
)

# Optional: Customize the layout further if needed
fig.update_layout(
    showlegend=True
)

fig.show()
